In [1]:
from ema_workbench.analysis import prim
from ema_workbench import load_results
from sklearn.decomposition import PCA
from sklearn import preprocessing
from problem_formulation import get_model_for_problem_formulation
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
experiments, outcomes = load_results("data/generated/genscen_Gorssel_50000_07-06-2021-05-33-45.tar.gz")
dike_model, planning_steps = get_model_for_problem_formulation("Overijssel")
uncertainties = [uncertainty.name for uncertainty in dike_model.uncertainties]
x = experiments[uncertainties]

In [3]:
# Prim takes a couple of minutes for 50 thousand, so lets opt for 30 thousand.

In [4]:
outcomes_df = pd.DataFrame(outcomes)

In [5]:
prim_algs = {}
for outcome in outcomes_df.columns:
    prim_algs[outcome] = prim.Prim(x, outcomes[outcome] > 0, threshold=0.9)
    print(outcome)

Difference in Expected Annual Damage Gorssel-Deventer
Expected Annual Damage Gorssel
Difference in Expected Number of Deaths Gorssel-Deventer


In [ ]:
boxes = {}
for key in outcomes_df.columns:
    boxes[key] = prim_algs[key].find_box()

In [ ]:
for key, box in boxes.items():
    print(key)
    box.show_tradeoff()
    plt.show()

In [ ]:
box_id=20

In [ ]:
boxes["Expected Annual Damage Gorssel"].inspect(box_id)
boxes["Expected Annual Damage Gorssel"].inspect(box_id, style='graph')
plt.show()

In [ ]:
boxes["Expected Annual Damage Gorssel"].select(box_id)
boxes["Expected Annual Damage Gorssel"].show_pairs_scatter()

In [ ]:
lims = boxes["Expected Annual Damage Gorssel"].box_lims[box_id]

In [ ]:
# so now we just need to get scenarios that meet these ranges

In [ ]:
conditions = []

for column in lims.columns:
    if isinstance(lims[column][0], set):
        conditions.append(experiments[column].isin(lims[column][0]))
    else:
        conditions.append(experiments[column] >= lims[column][0])
        conditions.append(experiments[column] <= lims[column][1])

df = pd.DataFrame(conditions).T

In [ ]:
experiments.iloc[df[(df== False).any(axis=1)].index]